# Kernel Sparsity via Pruning Tutorial

Neural networks, in general, are very overparamatarized for given tasks (ie the number of parameters far exceeds the number of training points) yet still they [generalize very well](https://arxiv.org/abs/1611.03530). This flies against conventional wisdom where overparamatarizing a model will lead to overfitting and putting theory behind this empirical evidence is a very active area of research.

Additionally, [early on](http://yann.lecun.com/exdb/publis/pdf/lecun-90b.pdf) it was discovered that large numbers of weights in neural networks could be pruned away (set to 0) without affecting the loss and in most cases actually improving the generalization capability of the network. This work was reinvigorated with Song Han's [2015 paper](https://arxiv.org/abs/1510.00149) in pursuit of compressing model size for mobile applications. This has resulted in numerous papers coming out on the topic of weight pruning, filter pruning, channel pruning, and ultimately block pruning. [This paper](https://arxiv.org/abs/1902.09574) out of Google gives a good overview of the current state of sparsity.

Given that models are very overparamatarized and large numbers of weights can be effectively pruned away, what does this leave us with? Well intuitively, then, we can think of pruning as performing an [architecture search](https://openreview.net/pdf?id=rJlnB3C5Ym) within this large, traditionally fixed weight space. What was originally important in the dense model was representing a large number of pathways for optimization. We can then effectively remove the unused pathways in the optimization space with a fine toothed comb (post training).

Well what does pruning get us? We now have a model with a lot of multiplications by zero that we don't need to run. If we're smart about how we do structure this compute (a surprisingly tricky problem), we can now run the model much faster than ever thought possible! That's where the [Neural Magic](http://neuralmagic.com/) engine can help us.

This tutorial provides a step by step walk through for pruning an already trained (dense) model. Specifically it is set up to work with the model trained in our [model training tutorial](model_training.ipynb), but it can be changed to support other models/datasets as needed:
1. Dataset selection
2. Model selection and loading
3. Pruning setup
4. Recalibration using pruning

In [1]:
import sys
import os

print('Python %s on %s' % (sys.version, sys.platform))

package_path = os.path.abspath(os.path.join(os.path.expanduser(os.getcwd()), os.pardir))
print(package_path)

"""
Adding the path to the neuralmagic-pytorch extension to the path so it isn't necessary to have it installed
"""
sys.path.extend([package_path])

print('Added current package path to sys.path')
print('Be sure to install from requirements.txt and pytorch separately')


Python 3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]] on linux
/home/mark/neuralmagic/Shared/neuralmagicml-pytorch
Added current package path to sys.path
Be sure to install from requirements.txt and pytorch separately


## Dataset Selection

We are using fast.ai's [Imagenette dataset](https://github.com/fastai/imagenette) provided under the [Apache License 2.0](https://github.com/fastai/imagenette/blob/master/LICENSE) as the default dataset. The original authors, much like ourselves, were interested in a dataset that has similar properties to more complicated datasets such as the Imagenet dataset but one that would allow rapid iterations. It includes 10 of the easiest classes out of the Imagenet 1000 dataset: tench, English springer, cassette player, chain saw, church, French horn, garbage truck, gas pump, golf ball, parachute. If you are interested in visualizing the properties in this dataset see our [model training tutorial](model_training.ipynb) which also gives a more in depth breakdown for what batch size to use and the dataset splits.

The dataset can easily be changed to the desired dataset in the code given.

Below we will need to fill in the dataset path, train batch size, and test batch size.

In [2]:
import ipywidgets as widgets
import torch

print('\nEnter the local path where the dataset can be found')

dataset_text = widgets.Text(value='', placeholder='Enter local path to dataset', description='Dataset Path')
display(dataset_text)

print('\nChoose the batch size to run through the model during train and test runs')
print('(be sure to press enter if/after inputting manually)')
train_batch_size_slider = widgets.IntSlider(
    value=64, min=1, max=256, step=1, description='Train Batch Size:'
)
display(train_batch_size_slider)
test_batch_size_slider = widgets.IntSlider(
    value=64 if torch.cuda.is_available() else 1, min=1, max=256, step=1, description='Test Batch Size:'
)
display(test_batch_size_slider)



Enter the local path where the dataset can be found


Text(value='', description='Dataset Path', placeholder='Enter local path to dataset')


Choose the batch size to run through the model during train and test runs
(be sure to press enter if/after inputting manually)


IntSlider(value=64, description='Train Batch Size:', max=256, min=1)

IntSlider(value=64, description='Test Batch Size:', max=256, min=1)

In [3]:
from neuralmagicML.datasets import ImagenetteDataset, EarlyStopDataset
from torch.utils.data import Dataset, DataLoader

dataset_root = os.path.abspath(os.path.expanduser(dataset_text.value.strip()))
print('\nLoading dataset from {}'.format(dataset_root))

if not os.path.exists(dataset_root):
    raise Exception('Folder must exist for dataset at {}'.format(dataset_root))
    
train_batch_size = train_batch_size_slider.value
test_batch_size = test_batch_size_slider.value

print('\nUsing train batch size of {} and test batch size of {}\n'
      .format(train_batch_size, test_batch_size))
    
train_dataset = ImagenetteDataset(dataset_root, train=True, rand_trans=True)
train_data_loader = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True, num_workers=4)
print('train dataset created: \n{}\n'.format(train_dataset))

val_dataset = ImagenetteDataset(dataset_root, train=False, rand_trans=False)
val_data_loader = DataLoader(val_dataset, batch_size=train_batch_size, shuffle=False, num_workers=4)
print('validation test dataset created: \n{}\n'.format(val_dataset))

train_test_dataset = EarlyStopDataset(ImagenetteDataset(dataset_root, train=True, rand_trans=False),
                                      early_stop=len(val_dataset))
train_test_data_loader = DataLoader(train_test_dataset, batch_size=train_batch_size, shuffle=False, num_workers=4)
print('train test dataset created: \n{}\n'.format(train_test_dataset))



Loading dataset from /home/mark/datasets/imagenette

Using train batch size of 256 and test batch size of 256

already downloaded imagenette of size ImagenetteSize.s160
train dataset created: 
Dataset ImagenetteDataset
    Number of datapoints: 12894
    Root location: /home/mark/datasets/imagenette/imagenette-160/train

already downloaded imagenette of size ImagenetteSize.s160
validation test dataset created: 
Dataset ImagenetteDataset
    Number of datapoints: 500
    Root location: /home/mark/datasets/imagenette/imagenette-160/val

already downloaded imagenette of size ImagenetteSize.s160
train test dataset created: 
Dataset ImagenetteDataset
    Number of datapoints: 500
    Root location: /home/mark/datasets/imagenette/imagenette-160/train



## Model Selection and Loading

For this exercise we'll create the standard [ResNet50 model](https://arxiv.org/abs/1512.03385) and in addition we will load the pretrained weights from our [model training tutorial](model_training.ipynb)

If you changed the dataset in the above cell, then we'll need to update the number of classes to create the model appropriately as well as loading your own pretrained weights. Additionally the model can be changed out completely to work with your specific use case.

Additionally, run the code block and select the device to run on before continuing. cpu runs in the pytorch cpu framework and cuda runs on an attached GPU.


In [4]:
import glob
import datetime
from neuralmagicML.models import resnet50, load_model

num_classes = 10
# TODO: change this to load pretrained weights from our cloud
model = resnet50(num_classes=num_classes)
model_id = '{}-{}'.format(model.__class__.__name__,
                          datetime.datetime.today().strftime('%Y-%m-%d-%H:%M:%S')
                              .replace('-', '.').replace(':', '.'))
pretrained_paths = [path for path in glob.glob('ResNet*.pth')]
pretrained_path = pretrained_paths[0]
load_model(pretrained_path, model)
print('Created model {}'.format(model.__class__.__name__))

print('\nChoose the device to run on')
device_choice = widgets.ToggleButtons(
    options=['cuda', 'cpu'] if torch.cuda.is_available() else ['cpu'],
    description='Device'
)
display(device_choice)


Created model ResNet

Choose the device to run on


ToggleButtons(description='Device', options=('cuda', 'cpu'), value='cuda')

## Pruning Setup

Informally, sparsity is the degree in which a tensor is comprised of zeros. More formally:

let $N^i$ be the total number of elements in a (e.g. weight) tensor $W_i$ and let $N^i_z$ be the number of elements which are zero-valued within that tensor.

The sparsity level associated with that tensor is then defined as $s_i \triangleq \dfrac{N^i_z}{N^i}$ 

Our goal now is to maximize the sparsity of each weight tensor within the model while preserving the accuracy of the densely trained version. The most robust and effective approach to this so far has been magnitude pruning. To frame the problem, let's say our goal is to go from an intial sparsity $s^i_{init}$ to a final sparsity $s^i_{final}$. We could start with changing out our usual $L_2$ weight regularization with $L_1$ [($L_2$ vs $L_1$)](https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c) in our cost function. Following through, we would find that we did, in fact, introduce a few zeros into our weights. We created a direct pathway within the optimization problem such that the model is incentivized to reduce the magnitude of unimportant weights to 0 (unimportant are defined as weights that do not significantly contribute to the loss function). This is hard to balance, though. For example, how do we determine the proper weighting between the loss function and the regularization such that we reach a desired sparsity without sacrificing accuracy? Also, empirically we find that the model will settle in local minima thus failing to further reduce weights to zero.

We can take this general idea to more of an extreme, though. Based on the previous thought experiment, it's reasonable to make an assumption that weights near zero are likely not important to the final loss function as well. Taking this even further, we could say that the smallest values within a given weight tensor are the ones least likely to affect our loss function. This is, of course, assuming that we've trained long enough to reach a stable point where our neural networks cost function has minimized unimportant weights. Naturally, then, we could apply a schedule where we prune away a small number of smallest weights for every $M$ training steps. This is exactly what 'magnitude pruning' does. 

Below, we go through a UI to set up a magnitude pruning schedule for our model. While pruning, some layers within a model will be more sensitive than others. A general rule is that the initial input layer and the final output layer are the most sensitive as they are an absolute bottleneck to the information flow. Because of this, we offer up a UI capable of creating very intricate schedules to the point that each layer could be pruned with a different schedule to a different final sparsity. In general, though, we can get by with ignoring the initial layer, pruning the final layer minimally (this is done to boost test accuracy for the pruned model), and prune all other layers equally. At Neural Magic we are actively working on automating this process to find the best possible pruning schedules to maximize performance while minimizing accuracy loss. 

The default for the below model will disable pruning for the initial and final layers and prune all other layers to 90% sparsity over the course of 10 epochs. The options available are described below:
- Prune Epochs: number of epochs to apply the pruning over
- Add New Group: add a new pruning group to the UI, used to prune selected layers to different sparsity levels at different rates
- Delete Current Group: remove the current group from the pruning schedule
- Tabs: the pruning groups setup so far, click to switch between
- Sparsity: a range slider where the left value defines the sparsity level to initially start pruning at and the right value represents the final sparsity level
- Start Epoch: the epoch to start pruning the selected layers at the initial sparsity
- End Epoch: the epoch to finish pruning the selected layers to the final sparsity
- Update Freq: the update frequency, in epochs, at which to prune the layers; ie 1.0 will prune every epoch
- Selectable Layers: a dropdown of the layers available in the model that can be pruned along with their FLOPS and param counts; select the desired layers to prune using the checkboxes

In [5]:
from neuralmagicML.notebooks import KSModifierWidgets


device = device_choice.value
print('running on device {}'.format(device))

print('\ncreating kernel sparsity analyzer widgets (need to execute the model, so may take some time)...')
ks_widget, ks_modifiers = KSModifierWidgets.interactive_module(model, device, inp_dim=(1, 3, 224, 224))

# add first group for all layers and remove the input and final layers
ks_widget.add_group(
    init_start_sparsity=0.05, init_final_sparsity=0.85, init_enabled=True,
    init_start_epoch=0.0, init_end_epoch=30.0, init_update_frequency=1.0
)
ks_modifiers[0].layers.pop(0)
ks_modifiers[0].layers.pop()
ks_widget.update_from_modifiers()

# add second group with just the final layer to set a lower sparsity for it
ks_widget.add_group(
    init_start_sparsity=0.05, init_final_sparsity=0.6, init_enabled=True,
    init_start_epoch=0.0, init_end_epoch=30.0, init_update_frequency=1.0
)
ks_modifiers[1].layers = ks_modifiers[1].layers[-1:]
ks_widget.update_from_modifiers()

display(ks_widget)


running on device cuda

creating kernel sparsity analyzer widgets (need to execute the model, so may take some time)...


Box(children=(VBox(children=(HBox(children=(Button(description='Add New Group', style=ButtonStyle()), Button(d…

### Learning Rate Selection

With our magnitude pruning schedule complete, we need to define the hyperparameters for training while pruning. The most important of these is the learning rate. Too high and we will diverge from our initial dense solution. Too low and we will have to train for too many epochs. Below we run a learning rate sensitivity analysis from the [cyclic LR paper](https://arxiv.org/abs/1506.01186).

To run the sensitivity analysis we will begin training the model at a very small learning rate ($10^{-7}$) where the weight updates will be lost in floating point precision errors (ie we won't learn). After each batch we exponentially increase the learning rate until we reach a very high learning rate ($10^0$) where we are guaranteed to diverge from our trained model.

A flat region will be apparent in the graph starting from the left to some point at the right for a fully trained model. After this we reach a critical learning rate where the loss begins to rapidly rise. This is the critical point where we begin diverging from the local minimum in our optimization space. Using this information, we can find the optimal learning rate to use with an [SGD + nesterov momentum optimizer](https://towardsdatascience.com/stochastic-gradient-descent-with-momentum-a84097641a5d) while pruning. Ideally we want to pick a learning rate that is a little before the divergent behavior. In this way we can guarantee fast convergence of the model after each pruning step. 

A good default for the learning rate is given for the model and dataset used in this notebook. If changing either, you will need to update the learning rate.

In [6]:
import torch
from neuralmagicML.utils import lr_analysis, lr_analysis_figure, CrossEntropyLossWrapper
%matplotlib inline
import matplotlib.pyplot as plt

### optimizer definitions
momentum = 0.9
weight_decay = 1e-4
###

# print('\nrunning learning rate analysis...')
# batches_per_sample = round(500 / train_batch_size)  # make sure we have enough sample points per learning rate
# analysis = lr_analysis(model, device, train_data_loader, CrossEntropyLossWrapper(), batches_per_sample,
#                        init_lr=1e-7, final_lr=1e0, sgd_momentum=momentum, sgd_weight_decay=weight_decay)
# lr_analysis_figure(analysis)
# plt.show()

print('\nselect the learning rate')
lr_slider = widgets.FloatLogSlider(
    value=0.01, min=-7, max=1, step=0.01, description='Learning Rate:'
)
display(lr_slider)



select the learning rate


FloatLogSlider(value=0.01, description='Learning Rate:', max=1.0, min=-7.0, step=0.01)

### Post Pruning Hyperparameters

After pruning the model, we'll need to train for a few final epochs to recover any lost accuracy. A typical setup is to train with an [exponential decay learning rate](https://towardsdatascience.com/learning-rate-schedules-and-adaptive-learning-rate-methods-for-deep-learning-2c8f433990d1) schedule starting from our previously found learning rate: $LR_i=LR_{init}*\gamma^i$ where $LR_i$ is the learning rate used at epoch $i$. In doing this, we can find a local min with equivalent accuracy to the dense model provided we did not sparsify the model too much.

Given this, the necessary hyperparameters to determine are:
- Num Epochs - the number of epochs to train for in the finalizing stage
- Final LR - the learning rate we will converge to while finalizing

These parameters must be filled in below. We give defaults for the original model and dataset in this notebook. The defaults were found using previous intuition along with a few training runs. In general if accuracy is not recovering, then training for longer overall as well for longer times at higher learning rates (increasing update freq) will help.

In [7]:
print('\nselect the number of epochs to train for after pruning')
finalize_epochs_text = widgets.IntText(value=30, description='Num Epochs')
display(finalize_epochs_text)

print('\nselect the final learning rate')
lr_final_slider = widgets.FloatLogSlider(
    value=0.001, min=-7, max=1, step=0.01, description='Final LR:'
)
display(lr_final_slider)



select the number of epochs to train for after pruning


IntText(value=30, description='Num Epochs')


select the final learning rate


FloatLogSlider(value=0.001, description='Final LR:', max=1.0, min=-7.0, step=0.01)

## Recalibration Using Pruning

With the parameters properly setup, we now create a schedule for applying the parameters. This is done using the Neural Magic ML library. Specifically we create a `ScheduledModifierManager` that controls a list of two different classes: `GradualKSModifier` which performs magnitude pruning to the weights in the given layers and `LearningRateModifier` which applies the exponential learning rate in the training epochs after pruning.

We then create a `ScheduledOptimizer` giving it the model, an SGD optimizer, the `ScheduledModifierManager` we created, and the training datset size. Using all this information, the code can apply any schedule needed by capturing the `.step()` call, perform schedule updates, and then calling into the original optimizer. Additionally `epoch_start()` and `epoch_end()` should be called on the optimizer wrapper while training. We will see this use in the next code block. 

For the loss function we will use cross entropy as is standard for classification tasks. For additional metrics we will use the top 1 accuracy, ie did we predict the class correctly or not. We do this because there are only 10 classes available so top N accuracy is generally uninformative with so few classes. We use a custom wrapper class to organize the metrics and loss into one, callable class.

Finally, beyond the usual basic screen-printouts let's use tensorboard's nice logging capabilities. We'll primarily track scalars such as the loss and accuracy throughout training in this example. We use tensorboardX to log from pytorch into tensorboard.


In [8]:
import math
from torch import optim
from torch.nn import Conv2d, Linear
from tensorboardX import SummaryWriter

from neuralmagicML.sparsity import (
    LearningRateModifier, ScheduledModifierManager, ScheduledOptimizer, KSAnalyzerLayer
)
from neuralmagicML.utils import TopKAccuracy, CrossEntropyLossWrapper


model = model.to(device)

prune_epochs = max([modifier.end_epoch for modifier in ks_modifiers])
finalizer_epochs = finalize_epochs_text.value
lr_init = lr_slider.value
lr_final = lr_final_slider.value
lr_gamma = (lr_final / lr_init) ** (1 / (finalizer_epochs - 1))
lr_modifier = LearningRateModifier(lr_class='ExponentialLR', lr_kwargs={'gamma': lr_gamma},
                                   start_epoch=prune_epochs, end_epoch=prune_epochs + finalizer_epochs,
                                   update_frequency=1)
modifier_manager = ScheduledModifierManager([lr_modifier, *ks_modifiers])
print('Created ScheduledModifierManager with exponential lr_modifier with gamma {} and {} ks modifiers'
      .format(lr_gamma, len(ks_modifiers)))

optimizer = optim.SGD(model.parameters(), lr_slider.value, momentum=momentum,
                      weight_decay=weight_decay, nesterov=True)
optimizer = ScheduledOptimizer(optimizer, model, modifier_manager, steps_per_epoch=len(train_dataset))
print('\nCreated scheudled optimizer with initial lr: {}, momentum: {}, weight decay: {}'
      .format(lr_slider.value, momentum, weight_decay))

loss = CrossEntropyLossWrapper(extras={'top1acc': TopKAccuracy(1)})
print('\nCreated loss wrapper\n{}'.format(loss))

logs_dir = os.path.abspath(os.path.expanduser(os.path.join('.', 'model_training_logs', model_id)))

if not os.path.exists(logs_dir):
    os.makedirs(logs_dir)

writer = SummaryWriter(logdir=logs_dir, comment='imagenette training')
print('\nCreated summary writer logging to \n{}'.format(logs_dir))


Created ScheduledModifierManager with exponential lr_modifier with gamma 0.9744600632908477 and 2 ks modifiers

Created scheudled optimizer with initial lr: 0.01, momentum: 0.9, weight decay: 0.0001

Created loss wrapper
CrossEntropyLossWrapper(Loss: cross_entropy; Extras: TopKAccuracy)

Created summary writer logging to 
/home/mark/neuralmagic/Shared/neuralmagicml-pytorch/notebooks/model_training_logs/ResNet-2019.08.11.11.59.07


### Applying Pruning Schedule

Below we go through a standard training and testing cycle in pytorch. 

for number of epochs required for pruning and finalizing:
train model over full training dataset (one epoch); update weights
test model over full validation dataset; no weight update
test model over sampled training dataset; no weight update

In the below code blocks, we create self-contained convenience functions for running the train and testing loops. These convenience functions are then called to train the model. At the end of the script we save the trained model in our current location with the date included as well as the final validation accuracy in the name.

Note, we have additionally created a logs directory which, in combination with tensorboard, can be used to visualize the progress of the model. To launch tensorboard use the following command from within the notebooks directory: 'tensorboard --logdir model_training_logs --port 6006' Now you will have an interactive dashboard running on http://localhost:6006

We additionally create analyzers for the kernel sparsity of all convs and kernel layers that is logged to tensorboard for visualization of the sparsity of each layer within the model. Also, we call `epoch_start()` and `epoch_end()` on the optimizer wrapper as mentioned above.

Finally, we save the final result next to this notebook under the model id given earlier. Happy pruning!


In [9]:
from tqdm import tqdm


def test_epoch(model, data_loader, loss, device, epoch):
    model.eval()
    results = {}
    
    with torch.no_grad():
        for batch, (*x_feature, y_lab) in enumerate(tqdm(data_loader)):
            y_lab = y_lab.to(device)
            x_feature = tuple([dat.to(device) for dat in x_feature])
            batch_size = y_lab.shape[0]
            y_pred = model(*x_feature)
            losses = loss(x_feature, y_lab, y_pred)
            
            for key, val in losses.items():
                if key not in results:
                    results[key] = []
                result = val.detach_().cpu()
                result = result.repeat(batch_size)
                results[key].append(result)
                
    return results

def train_epoch(model, data_loader, optimizer, loss, device, epoch, writer):
    model.train()
    init_batch_size = None
    batches_per_epoch = len(data_loader)
    
    for batch, (*x_feature, y_lab) in enumerate(tqdm(data_loader)):
        y_lab = y_lab.to(device)
        x_feature = tuple([dat.to(device) for dat in x_feature])
        batch_size = y_lab.shape[0]
        if init_batch_size is None:
            init_batch_size = batch_size
        optimizer.zero_grad()
        y_pred = model(*x_feature)
        losses = loss(x_feature, y_lab, y_pred)
        losses['loss'].backward()
        optimizer.step(closure=None)
        
        step_count = init_batch_size * (epoch * batches_per_epoch + batch)
        for _loss, _value in losses.items():
            writer.add_scalar('Train/{}'.format(_loss), _value.item(), step_count)
            writer.add_scalar('Train/Learning Rate', optimizer.learning_rate, step_count)
            
print('Recalibrating model for kernel sparsity...')

analyzed_layers = KSAnalyzerLayer.analyze_layers(
    model, layers=[name for name, mod in model.named_modules()
                   if isinstance(mod, Conv2d) or isinstance(mod, Linear)]
)
            
for epoch in tqdm(range(math.ceil(modifier_manager.max_epochs))):
    print('Starting epoch {}'.format(epoch))
    optimizer.epoch_start()
    train_epoch(model, train_data_loader, optimizer, loss, device, epoch, writer)
    
    print('Completed training for epoch {}, testing validation dataset'.format(epoch))
    val_losses = test_epoch(model, val_data_loader, loss, device, epoch)
    for _loss, _values in val_losses.items():
        _value = torch.mean(torch.cat(_values))
        last_value = _value
        writer.add_scalar('Test/Validation/{}'.format(_loss), _value, epoch)
        print('{}: {}'.format(_loss, _value))
        
    print('Completed testing validation dataset for epoch {}, testing training dataset'.format(epoch))
    train_losses = test_epoch(model, train_test_data_loader, loss, device, epoch)
    for _loss, _values in train_losses.items():
        _value = torch.mean(torch.cat(_values))
        writer.add_scalar('Test/Training/{}'.format(_loss), _value, epoch)
        print('{}: {}'.format(_loss, _value))
        
    optimizer.epoch_end()
    
    for ks_layer in analyzed_layers:
        writer.add_scalar('Kernel Sparsity/{}'.format(ks_layer.name), ks_layer.param_sparsity.item(), epoch + 1)
        
    print('Completed testing training dataset for epoch {}'.format(epoch))
    
pruned_save_path = os.path.abspath(os.path.expanduser(os.path.join('.', '{}.pth'.format(model_id))))
print('Finished pruning, saving model to {}'.format(pruned_save_path))
save_model(pruned_save_path, model, optimizer, epoch)
print('Saved model')


  0%|          | 0/120 [00:00<?, ?it/s]

Recalibrating model for kernel sparsity...
Starting epoch 0


/home/mark/neuralmagic/Shared/neuralmagicml-pytorch/neuralmagicML/sparsity/kernel/mask.py:193: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than tensor.new_tensor(sourceTensor).
  self._mask_tensor = self._param.data.new_tensor(self._mask_tensor)

  0%|          | 0/2 [00:00<?, ?it/s]

Completed training for epoch 0, testing validation dataset



  0%|          | 0/2 [00:00<?, ?it/s]

...


  0%|          | 0/2 [00:00<?, ?it/s]

loss: 0.3852517306804657
top1acc: 89.1999740600586
Completed testing validation dataset for epoch 119, testing training dataset



100%|██████████| 120/120 [58:19<00:00, 29.14s/it]

loss: 0.1325506567955017
top1acc: 95.5999755859375
Completed testing training dataset for epoch 119
Finished pruning, saving model to /home/mark/neuralmagic/Shared/neuralmagicml-pytorch/notebooks/ResNet-2019.08.11.11.59.07.pth


NameError: name 'save_model' is not defined